In [1]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

/home/agbe/MAG/CentralityFairness/Evaluations/lenskit/util.py:45: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass([
/home/agbe/MAG/CentralityFairness/Evaluations/lenskit/matrix.py:50: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass({
/home/agbe/MAG/CentralityFairness/Evaluations/lenskit/algorithms/funksvd.py:19: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numb

In [3]:
import numpy as np
import pandas as pd
import math
import pickle

In [4]:
# read input
dataset = os.path.join(currentdir, 'centralities.csv')
centralities = pd.read_csv(dataset, sep=';') # and more?? 
centralities.head()


,Author,Gender,MAC,PageRank,Katz,Degree
0,Agnes,1,1.0,5,5,6
1,Lasse,0,0.9,6,3,5
2,Kim,-1,1.0,1,1,1


In [5]:
# sort input, save top k only
def topK(k, centrality, data, removeUnknownGender):
    # check if centrality is valid?
    sortedCentrality = data.sort_values(by = centrality, ascending=False)
    if (removeUnknownGender):
        sortedCentrality = sortedCentrality.query('Gender != -1')
        
    renamed = sortedCentrality.rename(columns = {'Gender': 'item', centrality: 'rank'}, inplace=False)
    return renamed.head(k)

topK(k = 10, centrality='PageRank', data = centralities, removeUnknownGender=True)

,Author,item,MAC,rank,Katz,Degree
1,Lasse,0,0.9,6,3,5
0,Agnes,1,1.0,5,5,6


In [7]:
# evaluate rankings 
from lenskit.metrics.topnFair import *

calculate_equal_ex(recs=topK(2, 'Katz', centralities, False), protected_group=[0])





0.3562071871080222 1 0.38685280723454163 1


0.9207822211616017